LSTM 기반의 삼성전자 주가 예측 예제
 - yahoo finance 에서 데이터 다운로드 후 3일(3MA), 5일(5MA) 가격이평선 추가

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

데이터 불러오기

In [ ]:
raw_df = pd.read_csv('data/005930.KS.csv')  # yahoo finance 로부터 데이터 다운로드
raw_df.head()

In [ ]:
plt.title('SAMSUNG ELECTRONIC STCOK PRICE')
plt.ylabel('price')
plt.xlabel('period')
plt.grid()

plt.plot(raw_df['Adj Close'], label='Adj Close')
plt.show()

데이터 전처리 (Missing Data 처리, 정규화 등)

In [ ]:
# 통계정보 확인
raw_df.describe()

In [ ]:
# Missing Data 확인
raw_df.isnull().sum()

In [ ]:
# 최소값이 0 인 column 체크

for col in raw_df.columns:

    if raw_df[col].min() == 0:
        col_name = col
        print(col_name, type(col_name))

In [ ]:
raw_df.loc[raw_df['Volume']==0]

In [ ]:
# 각 column에 0 몇개인지 확인

for col in raw_df.columns:
    missing_rows = raw_df.loc[raw_df[col]==0].shape[0]
    print(col + ': ' + str(missing_rows))

In [ ]:
# 먼저 0 을 NaN 으로 바꾼후, Missing Data 처리
raw_df['Volume'] = raw_df['Volume'].replace(0, np.nan)

# 각 column에 0 몇개인지 확인
for col in raw_df.columns:
    missing_rows = raw_df.loc[raw_df[col]==0].shape[0]
    print(col + ': ' + str(missing_rows))

In [ ]:
# missing data 확인
raw_df.isnull().sum()

In [ ]:
raw_df.isnull().any()

In [ ]:
raw_df.loc[raw_df['Open'].isna()]

In [ ]:
# missing data 처리

raw_df = raw_df.dropna()
raw_df.isnull().sum()

In [ ]:
# 정규화 (Date 제외한 모든 수치부분 정규화)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scale_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', '3M', '5M', 'Volume']

scaled_df = scaler.fit_transform(raw_df[scale_cols])

scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)

print(scaled_df)

주가예측을 위해 3MA, 5MA, Adj Close 항목을 feature 선정
 - 정답은 Adj Close 선정
 - 시계열 데이터를 위한 window_size = 40 선정

In [ ]:
# 입력 파라미터 feature, label => numpy type

def make_sequene_dataset(feature, label, window_size):

    feature_list = []      # 생성될 feature list
    label_list = []        # 생성될 label list

    for i in range(len(feature)-window_size):
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])

    return np.array(feature_list), np.array(label_list)

In [ ]:
# feature_df, label_df 생성

feature_cols = [ '3M', '5M', 'Adj Close' ]
label_cols = [ 'Adj Close' ]

feature_df = pd.DataFrame(scaled_df, columns=feature_cols)
label_df = pd.DataFrame(scaled_df, columns=label_cols)

In [ ]:
feature_df.head()

In [ ]:
label_df.head()

In [ ]:
# DataFrame => Numpy 변환

feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

print(feature_np.shape, label_np.shape)

시계열 데이터 생성 (make_sequence_dataset)

In [ ]:
window_size = 40

X, Y = make_sequene_dataset(feature_np, label_np, window_size)

print(X.shape, Y.shape)

학습데이터, 테스트데이터 생성

In [ ]:
# train, test 분리

split = int(len(X)*0.95)
#split = -200

x_train = X[0:split]
y_train = Y[0:split]

x_test = X[split:]
y_test = Y[split:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

모델 구축 및 컴파일

In [ ]:
# model 생성

model = Sequential()

model.add(LSTM(128, activation='tanh', input_shape=x_train[0].shape))

model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

model.summary()

모델 학습 (EarlyStopping 적용)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10)

model.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          epochs=300, batch_size=16,
          callbacks=[early_stop])

In [ ]:
pred = model.predict(x_test)
print(pred[0], y_test[0])

예측을 통한 정답과의 비교 
(오차계산 MAPE 사용, 평균절대값백분율오차)

In [ ]:

plt.figure(figsize=(12, 6))
plt.title('3MA + 5MA + Adj Close, window_size=40')
plt.ylabel('adj close')
plt.xlabel('period')
plt.plot(y_test, label='actual')
plt.plot(pred, label='prediction')
plt.grid()
plt.legend(loc='best')

plt.show()

In [ ]:
# 평균절대값백분율오차계산 (MAPE)

print( np.sum(abs(y_test-pred)/y_test) / len(x_test) )